## echo server

## Test server

In [1]:
####### INIT ######################################################################################

from context import rmlib
from time import sleep

import numpy as np
import rmlib

from rmlib.rmtools.asm_BT_lib.asm_tree_Basic import augment_RM
import rmlib.rmtools as rm
from config import robotConfig

robot = rmlib.robot.Robot(robotConfig)
augment_RM(robot)

#loads variables attached to this notebook
%store -r  


####### LIBS ######################################################################################

# Import Pytrees
import py_trees
#from py_trees.tests import Timeout_Success
from py_trees.composites import Sequence , Selector

from rmlib.rmtools.asm_BT_lib.asm_tree_Basic import *
from rmlib.rmtools.asm_BT_lib.asm_tree_FT_based import *

import math
from pmath import translate_pose, get_disance_between_poses, pose_components

from utils import is_matx_list
from math import radians

print( "\n##!## READY ##!##" )

Loaded: /home/nvidia/dev_rmstudio/rmlib/rmlib/rmtools Exists?: True
Loaded: /home/nvidia/dev_rmstudio/rmlib Exists?: True
Setting up robot please wait...
my_components: {'arm_config': {'max_joint_speed': 1.05, 'xmlrpc_port': '8003', 'default_linear_speed': 0.1, 'ip_address': '192.168.0.101', 'class_name': 'UR5', 'max_joint_accel': 1.4, 'default_joint_accel': 0.8, 'module_name': 'arms.ur5', 'default_joint_speed': 0.7, 'default_linear_accel': 0.8, 'max_linear_accel': 1.2, 'max_linear_speed': 0.25}, 'ft_config': {'dataHz': 50.0, 'ip_address': '192.168.0.100', 'max_workers': 10, 'filter': {'alpha': 0.012, 'beta': 0.035}, 'remote_ip': '', 'module_name': 'sensors.ftsensor_optoforce', 'remote_port': 10000, 'class_name': 'OptoForce', 'local_ip': '127.0.0.1', 'local_port': 20000}, 'hand_config': {'finger_length': 0.0415, 'finger_width_inner': 0.0, 'finger_depth': 0.014, 'module_name': 'hands.smarthand', 'class_name': 'SmartHand', 'finger_width_outer': 0.015}}
active_componetns: {'ft': ['ft_conf

In [20]:
""""
from context import rmlib
#Imports 
from time import sleep
import numpy as np
import rmlib
from rmlib.rmtools.asm_BT_lib.asm_tree_Basic import augment_RM
import rmlib.rmtools as rm
from config import robotConfig

robot = rmlib.robot.Robot( robotConfig )

augment_RM( robot )
"""

'"\nfrom context import rmlib\n#Imports \nfrom time import sleep\nimport numpy as np\nimport rmlib\nfrom rmlib.rmtools.asm_BT_lib.asm_tree_Basic import augment_RM\nimport rmlib.rmtools as rm\nfrom config import robotConfig\n\nrobot = rmlib.robot.Robot( robotConfig )\n\naugment_RM( robot )\n'

In [2]:
import socket
import numpy as np
import struct

bot='red'  #'blue'

#Start and Stop socket
sock_SnS = socket.socket(socket.AF_INET, socket.SOCK_STREAM)# Create a TCP/IP socket
HOST_SnS = '192.168.0.103'
PORT_SnS= 65490
server_address_SnS = (HOST_SnS, PORT_SnS)# Bind the socket to the port
print('Starting up SnS socket on {} port {}'.format(*server_address_SnS))
sock_SnS.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock_SnS.bind(server_address_SnS)
# Listen for incoming connections
sock_SnS.listen(1)



# Wait for a connection
print('waiting for SnS connection')
connection_SnS, client_address_SnS = sock_SnS.accept()

print('SnS connection from', client_address_SnS)
startmsg='start'
data1=startmsg.encode('ascii')    
connection_SnS.sendall(data1)
print('SnS start command sent to client')

#Data and command socket
sock_DC = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
HOST_DC  = '192.168.0.103'
PORT_DC = PORT_SnS-10

# Bind the socket to the port
server_address_DC  = (HOST_DC , PORT_DC )
print('Starting up DC socket on {} port {}'.format(*server_address_DC))
sock_DC.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock_DC.bind(server_address_DC)
sock_DC.listen(1)# Listen for incoming connections


try:
    while True:
        # Wait for a connection
        print('waiting for DC connection')
        connection_DC, client_address_DC = sock_DC.accept()

        print('DC connection from', client_address_DC)

        # Receive the data in small chunks and retransmit it
        while True:
            command = connection_DC.recv(64)
            print("command:",command)
            if command==b'end':
                connection_DC.close()
                print("Closing+shutting down DC connection")
                break
            elif command==b'obs':
                if bot=='blue':
                    print('sending pose data to DC client')
                    data=robot.arm.get_tcp_pose() 
                    wristft=robot.arm.get_tcp_force()
                    print(data)
                            #dat=bytes(data[i][j])#.encode('ascii')  arr.tolist()
                    posedatabyte=struct.pack('ffffffffffffffffffffff',
                                data[0][0],data[0][1],data[0][2],data[0][3],
                                data[1][0],data[1][1],data[1][2],data[1][3],
                                data[2][0],data[2][1],data[2][2],data[2][3],
                                data[3][0],data[3][1],data[3][2],data[3][3],
                                wristft[0], wristft[1],wristft[2],
                                wristft[3],wristft[4],wristft[5])
                    print(posedatabyte)
                    connection_DC.sendall(posedatabyte)   #was send  
                elif bot=='red':
                    print('sending pose data to DC client')
                    data=robot.arm.get_tcp_pose() 
                    print(data)
                            #dat=bytes(data[i][j])#.encode('ascii')  arr.tolist()
                    posedatabyte=struct.pack('ffffffffffffffff',
                                data[0][0],data[0][1],data[0][2],data[0][3],
                                data[1][0],data[1][1],data[1][2],data[1][3],
                                data[2][0],data[2][1],data[2][2],data[2][3],
                                data[3][0],data[3][1],data[3][2],data[3][3])
                    print(posedatabyte)
                    connection_DC.sendall(posedatabyte)   #was send  
            #else:
                #print('no data from', client_address_DC)
                #break
            elif (command==b'h' or command==b'j' or command==b'k' or command==b'l' or 
                command==b'u' or command==b'o'or command==b'y' or command==b'i' or 
                command==b'z' or command==b'x' or command==b'd' or command==b'c'or
                command==b'home' or command == b'fetch' or command == b'return' or command == b'reset' 
                  or command == b'' ):
                print('received {!r}'.format(command))
            else:
                xy=struct.unpack('dd',command)
                print("xmove=",xy[0])
                print("ymove=",xy[1])
        if command==b'end':
            break
    stopcommand = connection_SnS.recv(16)
    print('received {!r}'.format(stopcommand))
    if stopcommand==b'end':
        print("Closing+shutting down SnS connection")
        connection_SnS.close()                   
finally:
   
        print("Closing+shutting down SnS connection")
        connection_SnS.close()    
        print("Closing+shutting down DC connection")
        connection_DC.close()    

Starting up SnS socket on 192.168.0.103 port 65490
waiting for SnS connection
SnS connection from ('192.168.0.102', 44250)
SnS start command sent to client
Starting up DC socket on 192.168.0.103 port 65480
waiting for DC connection
DC connection from ('192.168.0.102', 52182)
command: b'u'
received b'u'
command: b'obs'
sending pose data to DC client
[[-0.99934309 -0.00726197  0.03550561 -0.13836779]
 [-0.00684194  0.99990534  0.01193738 -0.30214753]
 [-0.03558894  0.01168661 -0.99929818  0.24813912]
 [ 0.          0.          0.          1.        ]]
b'\xf3\xd4\x7f\xbf\xd7\xf5\xed\xbbTn\x11=I\xb0\r\xbeP2\xe0\xbb\xcc\xf9\x7f?\x03\x95C<\x15\xb3\x9a\xbe\xb5\xc5\x11\xbd6y?<\x01\xd2\x7f\xbf.\x18~>\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80?'
command: b'obs'
sending pose data to DC client
[[-0.99934309 -0.00726197  0.03550561 -0.13836779]
 [-0.00684194  0.99990534  0.01193738 -0.30214753]
 [-0.03558894  0.01168661 -0.99929818  0.24813912]
 [ 0.          0.          0.        

NameError: name 'stopcommand' is not defined

In [4]:
x=0
f=5
try:
    f> 3
except:
    print(Exception("Something went wrong") )
    print("testo")
    


finally:
    print("blah")

blah
